## Task

In this compulsory task you will clean the country column and parse the date column in the **store_income_data_task.csv** file.

In [1]:
import pandas as pd
import numpy as np
import fuzzywuzzy
from fuzzywuzzy import process
import chardet

In [2]:
# Load up store_income_data.csv
df_income = pd.read_csv("store_income_data_task.csv")
df_income.info()
df_income.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             1000 non-null   int64 
 1   store_name     1000 non-null   object
 2   store_email    413 non-null    object
 3   department     973 non-null    object
 4   income         1000 non-null   object
 5   date_measured  1000 non-null   object
 6   country        965 non-null    object
dtypes: int64(1), object(6)
memory usage: 54.8+ KB


,id,store_name,store_email,department,income,date_measured,country
0,1,"Cullen/Frost Bankers, Inc.",NaN,Clothing,$54438554.24,4-2-2006,United States/
1,2,Nordson Corporation,NaN,Tools,$41744177.01,4-1-2006,Britain
2,3,"Stag Industrial, Inc.",NaN,Beauty,$36152340.34,12-9-2003,United States
3,4,FIRST REPUBLIC BANK,ecanadine3@fc2.com,Automotive,$8928350.04,8-5-2006,Britain/
4,5,Mercantile Bank Corporation,NaN,Baby,$33552742.32,21-1-1973,United Kingdom


1. Take a look at all the unique values in the "country" column. Then, convert the column to lowercase and remove any trailing white spaces.

In [3]:
countries = df_income['country'].unique()
print(f"There are {len(countries)} unique countries")
countries

There are 77 unique countries


array(['United States/', 'Britain', ' United States', 'Britain/',
       ' United Kingdom', 'U.K.', 'SA ', 'U.K/', 'America',
       'United Kingdom', nan, 'united states', ' S.A.', 'England ', 'UK',
       'S.A./', 'ENGLAND', 'BRITAIN', 'U.K', 'U.K ', 'America/', 'SA.',
       'S.A. ', 'u.k', 'uk', ' ', 'UK.', 'England/', 'england',
       ' Britain', 'united states of america', 'UK/', 'SA/', 'SA',
       'England.', 'UNITED KINGDOM', 'America.', 'S.A..', 's.a.', ' U.K',
       ' United States of America', 'Britain ', 'England', ' SA',
       'United States of America.', 'United States of America/',
       'United States.', 's. africasouth africa', ' England',
       'United Kingdom ', 'United States of America ', ' UK',
       'united kingdom', 'AMERICA', 'America ',
       'UNITED STATES OF AMERICA', ' S. AfricaSouth Africa', 'america',
       'S. AFRICASOUTH AFRICA', 'Britain.', '/', 'United Kingdom.',
       'United States', ' America', 'UNITED STATES', 'sa',
       'United States

2. Note that there should only be three separate countries. Eliminate all variations, so that 'South Africa', 'United Kingdom' and 'United States' are the only three countries.

In [8]:
#Convert to lower case 
df_income['country'] = df_income['country'].str.lower()

# Remove trailing white spaces
df_income['country'] = df_income['country'].str.strip()

# Let us view the data
countries = df_income['country'].unique()
print(f"There are {len(countries)} unique countries")

# Function to replace rows in the provided column of the provided DataFrame
# that match the provided string above the provided ratio with the provided string
def replace_matches_in_column(df, column, string_to_match, min_ratio = 90):
    # get a list of unique strings
    strings = df[column].unique()
    
    # Get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(string_to_match, strings, 
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    # Only get matches with a ratio > 90
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]

    # Get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    # Replace all rows with close matches with the input matches 
    df.loc[rows_with_matches, column] = string_to_match
    
    # Let us know when the function is done
    print("All done!")

replace_matches_in_column(df=df_income, column='country', string_to_match="south africa")
replace_matches_in_column(df=df_income, column='country', string_to_match="united kingdom")
replace_matches_in_column(df=df_income, column='country', string_to_match="united states")

# 'South Africa', 'United Kingdom' and 'United States'

#Manually cleaning (Want "united kingdom" and anything else is wrong)
mistype_uk = ['britain','britain/','u.k.','u.k/','england','u.k','uk.',
              'england/','uk/','england.','britain.','uk']
#replace('find','replace with')
df_income.replace(mistype_uk,'united kingdom',inplace=True)

#Manually cleaning (Want "united states" and anything else is wrong)
mistype_usa = ['america','america/','united states of america','america.','united states of america.'
               'united states of america/','united states of america.','united states of america/']
df_income.replace(mistype_usa,'united states',inplace=True)

mistype_sa = ['sa','s.a','s.a/','sa.','sa/','s.a..','s. africasouth africa',
              's. africasouth africa/','s. africasouth africa.','s.a.','s.a./',]
df_income.replace(mistype_sa,'south africa',inplace=True)

mistype_other = ['','/','.']
df_income.country.fillna('other', inplace=True)
df_income.replace(mistype_other,'other',inplace=True)

df_income = df_income[df_income['country'] != 'other']
df_income.reset_index(drop=True, inplace=True)

countries = df_income['country'].unique()
print(f"There are {len(countries)} unique countries")

countries


There are 3 unique countries
All done!
All done!
All done!
There are 3 unique countries


C:\Users\Salman\AppData\Local\Temp\ipykernel_15216\3554970827.py:55: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_income.country.fillna('other', inplace=True)


array(['united states', 'united kingdom', 'south africa'], dtype=object)

After cleaning my dataset the results are here
- The amount of country we have right now is **1000**
- The amount of country 'other is **86**

Going to drop the 'other' which is about **8.6%** which I consider to be too significant but since the objective of the wants to only three country is 'South Africa', 'United Kingdom' and 'United States'

3. Create a new column called `days_ago` in the DataFrame that is a copy of the 'date_measured' column but instead it is a number that shows how many days ago it was measured from the current date. Note that the current date can be obtained using `datetime.date.today()`.